In [1]:
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix

In [2]:
a=pd.read_csv('./TSPA.csv', sep=';', header=None)
tspa=pd.DataFrame(data=a.values, columns=['x', 'y', 'val'])


In [3]:
coordinates=tspa[['x', 'y']].values

In [12]:
dist=distance_matrix(coordinates, coordinates)
dist=np.round(dist)
dist+=a

In [13]:
costs=tspa[['val']].values

In [14]:
cost_m=dist+costs

In [16]:
def get_random_solution(n: int):
    a=np.arange(0,n)
    np.random.shuffle(a)
    return a[:n//2]

In [21]:
def nn(starting_point: int, cost_matrix: np.ndarray):
    result_list=[starting_point]
    n = cost_matrix.shape[0]
    for x in range(n-1):
        distances=cost_matrix[result_list[-1]]
        indices=np.arange(0,n)
        di_pairs=[(d, i) for d, i in zip(distances,indices) if i not in result_list]
        nearest_neighbor = min(di_pairs)
        result_list.append(nearest_neighbor[1])
    return result_list